In [ ]:
from rich.console import Console
from dotenv import load_dotenv
import json 
from openai import OpenAI
load_dotenv(override=True)

In [ ]:
def show(text):
    try:
       Console().print(text)  
    except Exception:
        print(text)

        

In [ ]:
todos = []
completed = []

In [ ]:
def print_todos():
    for i, todo in enumerate(todos):
        if completed[i]:
            show(f"Todo #{i+1}: [green][strike]{todo}[/strike][/green]\n")
        else:
            show(f"Todo #{i+1}: {todo}\n")


In [ ]:
def create_todos(descriptions: list[str]):
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return print_todos()



In [ ]:
todos, completed = [], []

create_todos(["Buy groceries", "Finish the course", "Buy a new laptop"])

In [ ]:
def mark_complete(index: int, completion_notes: str):
    if index < 0 or index >= len(todos):
        return show("Invalid index")
    completed[index-1] = True
    Console().print(completion_notes)

    return print_todos()





In [39]:
mark_complete(2, "finished")

finished

Todo #1: Buy groceries

Todo #2: Finish the course

Todo #3: Buy a new laptop

In [ ]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [ ]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [ ]:
tools = [{"type": "function", "function": create_todos_json},
        {"type": "function", "function": mark_complete_json}]

In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
openai = OpenAI()
todos, completed = [], []

def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-5.2", messages=messages, tools=tools, reasoning_effort="none")
        finish_reason = response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
        show(response.choices[0].message.content)

In [ ]:
system_message = """
You are given a problem to solve by using yout todo tools to plan a ist of steps, then carrying out the steps one by one. 
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """
Two boats set out from the port in different directions. The first boat traveled 2 km north to point A, and the second boat traveled 3 km east to point B.
What is the distance between the boats at points A and B?
"""

messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]


In [40]:
todos, completed = [], []
loop(messages)

TypeError: create_todos() got an unexpected keyword argument 'descriptions'